<a href="https://colab.research.google.com/github/TripleBlackCat/masakhane-mt/blob/master/starter_notebook_Kimbundu_into_English_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masakhane - Reverse Machine Translation for African Languages (Using JoeyNMT)

> ## NB
>### - The purpose of this Notebook is to build models that translate African languages(target language) *into* English(source language). This will allow us to in future be able to make translations from one African language to the other. If you'd like to translate *from* English, please use [this](https://github.com/masakhane-io/masakhane-mt/blob/master/starter_notebook.ipynb) starter notebook instead.

>### - We call this reverse training because normally we build models that make translations from the source language(English) to the target language. But in this case we are doing the reverse; building models that make translations from the target language to the source(English)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "kmb" # Set target language to Kimbundu
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$tgt-$src-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (target_language, source_language, tag)

In [4]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/kmb-en-baseline


In [5]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 80 kB 4.1 MB/s 


In [6]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-kmb.xml.gz not found. The following files are available for downloading:

   1 MB https://object.pouta.csc.fi/OPUS-JW300/v1c/xml/en-kmb.xml.gz
 274 MB https://object.pouta.csc.fi/OPUS-JW300/v1c/xml/en.zip
  11 MB https://object.pouta.csc.fi/OPUS-JW300/v1c/xml/kmb.zip

 286 MB Total size
./JW300_latest_xml_en-kmb.xml.gz ... 100% of 1 MB
./JW300_latest_xml_en.zip ... 100% of 274 MB
./JW300_latest_xml_kmb.zip ... 100% of 11 MB


In [7]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2021-10-08 18:38:52--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.03s   

2021-10-08 18:38:52 (8.84 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2021-10-08 18:38:52--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-kmb.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 

In [8]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [9]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 3413/104046 lines since contained in test set.


,source_sentence,target_sentence
0,"December 1 , 2010",1 Ua Katatu Ua 2011
1,Who Inhabit the Spirit Realm ?,O Kuiala ku Diulu Kuene Muene Athu mu Nzumbi
2,FROM OUR COVER,TU SANGA - MU UÉ MILONGI ÍII


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [10]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np
from os import cpu_count
from functools import partial
from multiprocessing import Pool


# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

     |████████████████████████████████| 50 kB 3.1 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149863 sha256=868b333fa121fffe2f41a62b6dfa085c6e929780931bd93051f896f3eebe694e
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [12]:
# start_time = time.time()
# ### iterating over pandas dataframe rows is not recomended, let use multi processing to apply the function

# with Pool(cpu_count()-1) as pool:
#     scores = pool.map(partial(fuzzfilter, candidates=list(en_test_sents), pad=5), df_pp['source_sentence'])
# hours, rem = divmod(time.time() - start_time, 3600)
# minutes, seconds = divmod(rem, 60)
# print("done in {}h:{}min:{}seconds".format(hours, minutes, seconds))

# # Filter out "almost overlapping samples"
# df_pp = df_pp.assign(scores=scores)
# df_pp = df_pp[df_pp['scores'] < 95]

In [13]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
7 Like David , Jehovah ’ s people today try to be humble .
We want to stay up-to-date with the understanding of Bible truth so that we can share accurate knowledge with others .
This creates a bond of unity impossible to break .
It describes Mary ’ s visit from the angel Gabriel , her visit to her relative Elizabeth , and her words of praise to Jehovah .
( 1 Cor . 5 : 7 ; 11 : 23 ; Matt . 26 : 2 ) As the end of that Jewish day approached , Jesus was buried — before the start of Nisan 15 . * — Lev . 23 : 5-7 ; Luke 23 : 54 .
What , then , is the source of injustice ?
It does not become provoked . It does not keep account of the injury .
18 The first century was a unique time in history .
Jehovah was not going to save the things that people owned .
I did not know what he meant and did not recognize him .

==> train.kmb <==
7 Kala Davidi o mundu ua Jihova lelu , u sota ue kulondekesa o kulenduka .
Tua tokala ku mona se ki tua mu longa o milongi ia Bibidia iokulu , phala t



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [14]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .
# Install Pytorch with GPU support v1.7.1.
# JoeyNMT installs pytorch, the below version was resulting in errors
# ! pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Cloning into 'joeynmt'...
remote: Enumerating objects: 3224, done.
remote: Counting objects: 100% (273/273), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 3224 (delta 157), reused 206 (delta 134), pack-reused 2951
Receiving objects: 100% (3224/3224), 8.17 MiB | 15.58 MiB/s, done.
Resolving deltas: 100% (2185/2185), done.
Processing /content/joeynmt
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 90 kB 4.1 MB/s 
     |████████████████████████████████| 636 kB 38.1 MB/s 
     |████████████████████████████████| 392 kB 45.5 MB/s 
     |████████████████████████████████| 246 kB 43.6 MB/s

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [15]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data",target_language + source_language ) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$tgt$src/train.bpe.$src joeynmt/data/$tgt$src/train.bpe.$tgt --output_path joeynmt/data/$tgt$src/vocab.txt

# Some output
! echo "BPE Kimbundu Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$tgt$src/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.kmb    test.kmb	    train.en
dev.bpe.en	dev.kmb      test.en	     train.bpe.en   train.kmb
dev.bpe.kmb	test.bpe.en  test.en-any.en  train.bpe.kmb
bpe.codes.4000	dev.en	     test.bpe.kmb    test.kmb	    train.en
dev.bpe.en	dev.kmb      test.en	     train.bpe.en   train.kmb
dev.bpe.kmb	test.bpe.en  test.en-any.en  train.bpe.kmb
BPE Kimbundu Sentences
O ngu@@ b@@ u ya ku@@ xikana ( Tala o kaxi 12 - 14 )
O ka@@ pas@@ ete ka kubh@@ uluka ( Tala o kaxi 15 - 18 )
Nga mono kwila o athu a xikina dingi se a mona kwila ey@@ e wa zolo mwene o Bibi@@ dya , wa mu bhanga yoso i u tena phala ku a kwatekesa . ”
O xi@@ bhata ya nzumbi ikôla ( Tala o kaxi 19 - 20 )
Ni ki@@ kw@@ at@@ ek@@ esu kya Jihova tu tena kubh@@ ânga nê !
Combined BPE Vocab
⁀@@
‚
Û@@
Efe@@
especi@@
SONG@@
HAT
Gree@@
‛
ímbi


In [16]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.en	     test.bpe.kmb    test.kmb	    train.en
dev.bpe.en	dev.kmb      test.en	     train.bpe.en   train.kmb
dev.bpe.kmb	test.bpe.en  test.en-any.en  train.bpe.kmb


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [15]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_reverse_$tgt$src.yaml

2021-10-09 08:48:50,437 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-09 08:48:50,451 - INFO - joeynmt.data - Loading training data...
2021-10-09 08:48:51,308 - INFO - joeynmt.data - Building vocabulary...
2021-10-09 08:48:51,409 - INFO - joeynmt.data - Loading dev data...
2021-10-09 08:48:51,417 - INFO - joeynmt.data - Loading test data...
2021-10-09 08:48:51,427 - INFO - joeynmt.data - Data loaded.
2021-10-09 08:48:51,427 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-09 08:48:51,542 - INFO - joeynmt.model - Enc-dec model built.
2021-10-09 08:48:51,545 - INFO - joeynmt.training - Total params: 12143104
2021-10-09 08:48:51,546 - WARNING - joeynmt.training - `keep_last_ckpts` option is outdated. Please use `keep_best_ckpts`, instead.
2021-10-09 08:48:53,710 - INFO - joeynmt.helpers - cfg.name                           : kmben_reverse_transformer
2021-10-09 08:48:53,710 - INFO - joeynmt.helpers - cfg.data.src                       : kmb
2021-10-09

In [16]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/kmben_reverse_transformer/* "$gdrive_path/models/${src}${tgt}_reverse_transformer/"

cp: target 'masakhane/kmb-en-baseline/models/enkmb_reverse_transformer/' is not a directory


In [22]:
! $gdrive_path

zsh:1: permission denied: masakhane/kmb-en-baseline


In [23]:
# Output our validation accuracy
! cat "joeynmt/models/${tgt}${src}_reverse_transformer/validations.txt"

Steps: 1000	Loss: 103070.83594	PPL: 58.30863	bleu: 2.27720	LR: 0.00030000	*
Steps: 2000	Loss: 90314.13281	PPL: 35.25287	bleu: 5.05797	LR: 0.00030000	*
Steps: 3000	Loss: 81559.82812	PPL: 24.95872	bleu: 10.06160	LR: 0.00030000	*
Steps: 4000	Loss: 76023.91406	PPL: 20.06251	bleu: 13.63231	LR: 0.00030000	*
Steps: 5000	Loss: 72320.81250	PPL: 17.33590	bleu: 13.85980	LR: 0.00030000	*
Steps: 6000	Loss: 69462.28906	PPL: 15.48732	bleu: 16.08688	LR: 0.00030000	*
Steps: 7000	Loss: 67206.17969	PPL: 14.16858	bleu: 17.40593	LR: 0.00030000	*
Steps: 8000	Loss: 65259.01172	PPL: 13.12106	bleu: 18.05752	LR: 0.00030000	*
Steps: 9000	Loss: 63758.84766	PPL: 12.36714	bleu: 19.08144	LR: 0.00030000	*
Steps: 10000	Loss: 62352.35938	PPL: 11.69969	bleu: 19.62039	LR: 0.00030000	*
Steps: 11000	Loss: 60913.42969	PPL: 11.05410	bleu: 20.29941	LR: 0.00030000	*
Steps: 12000	Loss: 59930.32031	PPL: 10.63363	bleu: 20.93355	LR: 0.00030000	*
Steps: 13000	Loss: 58837.71094	PPL: 10.18507	bleu: 21.46749	LR: 0.00030000	*
Steps: 14

In [35]:
# Test our model
# ! python3 -m joeynmt test "joeynmt/models/kmben_reverse_transformer/config.yaml"
! python3 -m joeynmt test "models/kmben_reverse_transformer/config.yaml"

2021-10-09 11:08:19,763 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-09 11:08:19,763 - INFO - joeynmt.data - Building vocabulary...
2021-10-09 11:08:19,866 - INFO - joeynmt.data - Loading dev data...
2021-10-09 11:08:19,871 - INFO - joeynmt.data - Loading test data...
2021-10-09 11:08:19,882 - INFO - joeynmt.data - Data loaded.
2021-10-09 11:08:19,910 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 3600
2021-10-09 11:08:19,911 - INFO - joeynmt.prediction - Loading model from models/kmben_reverse_transformer/48000.ckpt
2021-10-09 11:08:22,072 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-09 11:08:22,190 - INFO - joeynmt.model - Enc-dec model built.
2021-10-09 11:08:22,217 - INFO - joeynmt.prediction - Decoding on dev set (data/kmben/dev.bpe.en)...
2021-10-09 11:08:37,680 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-10-09 11:08:37,680 - WARNING - sacrebleu - It looks like 